In [3]:
import importlib
import nanotools
importlib.reload(nanotools) # reload nanotools module
#import random
#random.seed(10)
import numpy as np
import pandas as pd
import plotly.express as px # Used for plotting
import plotly.graph_objects as go # Used for plotting
from plotly.subplots import make_subplots # Used for plotting
from sklearn import cluster # Used for clustering
import multiprocessing
from multiprocessing import Pool, cpu_count # used for parallel processing
from io import StringIO
import subprocess
import os
import plotly.io as pio
import plotly
import matplotlib.pyplot as plt # Use for plotting m6A frac and coverage plot
from matplotlib import cm # Use for plotting m6A frac and coverage plot
import sqlite3
import tqdm
import pysam

#import dask.dataframe as dd

# set renderer to vscode
plotly.offline.init_notebook_mode(connected=True)
pio.renderers.default = 'plotly_mimetype+notebook'
pd.options.display.max_rows = None
pd.set_option('display.max_columns', None)
# display count_df with no limits on rows
pd.set_option('display.max_rows', None)
# left align print
pd.set_option('display.max_colwidth', None)

In [4]:
### Bed file configurations:
sample_source = "chr_type" # "chr_type" or "type" or "chromosome"
sampleName = ["X","Autosome"] # "TES_q1" "strong_rex" "weak_rex" "type", "X", "Autosome"; Must be same number of unique values in selected bed rows.
chr_type_selected = ["X","Autosome"] # 'X' or "Autosome"
type_selected = ["TSS_q4","TSS_q3","TSS_q2","TSS_q1"] #her-1_TSS","fem-1_TSS","fem-2_TSS","fem-3_TSS","sex-1_TSS",
# her-1_TSS/TES/FULL | TES_q1-4 | #TSS_q1-4 | strong/weak rex | whole_chr | 200kb_region | 50kb_region | center_DPY27_chip_albretton | gene_q1-q4 | MEX_motif | center_SDC2_chip_albretton | center_SDC3_chip_albretton |
max_regions = 0 # max regions to consider; 0 = full set;
chromosome_selected = ["CHROMOSOME_I", "CHROMOSOME_II", "CHROMOSOME_III", "CHROMOSOME_IV","CHROMOSOME_V","CHROMOSOME_X"]
strand_selected = ["+","-"] #+ and/or -
select_opp_strand = True #If you want to select both + and - strands for all regions set to True
down_sample_autosome = True # If you want to downsample autosome genes to match number of X genes set to True
if chr_type_selected == ["X"]:
    down_sample_autosome = False
bed_file = "/Data1/reference/tss_tes_rex_combined_v11_WS235.bed"
bed_window = 1000 # +/- around bed elements.
mods = "a" # {A,CG,A+CG}
if sample_source == "chr_type":
    selection = chr_type_selected
if sample_source == "type":
    selection = type_selected
if sample_source == "chromosome":
    selection = chromosome_selected

In [5]:
### BAM Configurations
m6A_thresh = 181 #default is 129 = 50%; 181=70%; 194=75%; 207 = 80%; 232 = 90%
mC_thresh = 129 #default is 129

# modkit is used for aggregating methylation data from .bam files
# https://nanoporetech.github.io/modkit/quick_start.html
modkit_path = "/Data1/software/modkit/modkit"

### IMPORT BAM FILES AND METADATA FROM CSV FILE
input_metadata = pd.read_csv("/Data1/seq_data/combined_fiber_dimelo_8_30_23/input_metadata_8_30_23.txt", sep="\t", header=0)
# Options: N2_fiber; SDC2_degron_fiber; SDC2_degron_bg; N2_bg; N2-DPY27_dimelo_pAHia5
analysis_cond = ["N2_fiber","SDC2_degron_fiber"]#,"N2_young"]
# Set bam_files equal to list of items in column bam_files where conditions == N2_fiber
bam_files = input_metadata[input_metadata["conditions"].isin(analysis_cond)]["bam_files"].tolist()
conditions = input_metadata[input_metadata["conditions"].isin(analysis_cond)]["conditions"].tolist()
exp_ids = input_metadata[input_metadata["conditions"].isin(analysis_cond)]["exp_id_date"].tolist()
bam_fracs = len(bam_files)*[1] # For full .bam set to = 1
sample_indices = list(range(len(bam_files)))
output_stem = "/Data1/seq_data/combined_fiber_dimelo_8_30_23/"
thresh_list=len(bam_files)*[m6A_thresh/258] # For R10 flow cells use 0.5; for R9 flow cells use 0.9
file_prefix = "aug_27_"
"""
if sample_source == "chr_type":
    selection = chr_type_selected
if sample_source == "type":
    selection = type_selected
if sample_source == "chromosome":
    selection = chromosome_selected
if sample_source == "condition":
    selection = analysis_cond"""

'\nif sample_source == "chr_type":\n    selection = chr_type_selected\nif sample_source == "type":\n    selection = type_selected\nif sample_source == "chromosome":\n    selection = chromosome_selected\nif sample_source == "condition":\n    selection = analysis_cond'

In [6]:
# Filter input bed_file based on input parameters (e.g. chromosome, type, strand, etc.)
# Function saves a new filtered bed file to the same folder as the original bed file
# called temp_do_not_use_"type".bed
importlib.reload(nanotools)
new_bed_files=nanotools.filter_bed_file(
    bed_file,
    sample_source,
    selection,
    chromosome_selected,
    chr_type_selected,
    type_selected,
    strand_selected,
    max_regions,
    bed_window
)

modkit_bed_name = "modkit_temp.bed"
modkit_bed_df = nanotools.generate_modkit_bed(new_bed_files, down_sample_autosome, select_opp_strand,modkit_bed_name)
nanotools.display_sample_rows(modkit_bed_df, 5)

# Subsample bam based on bam_frac, used to accelerate testing
# if bam_frac = 1 will use original bam files, otherwise will save new subsampled bam files to output_stem.
args_list = [(bam_file, condition, bam_frac, sample_index, output_stem) for bam_file, condition, bam_frac, sample_index in zip(bam_files,conditions,bam_fracs,sample_indices)]
new_bam_files=[]
if __name__ == "__main__":
    with Pool() as pool: #processes=1
        new_bam_files = pool.starmap(nanotools.subsample_bam, args_list)
    print("Program finished!")

print(new_bam_files)

| temp_bed | first 5, random 5 and last 5 out of total 2289 rows.


,chromosome,start,end,strand,type,chr-type
0,CHROMOSOME_X,2623,4623,+,TSS_q2,X
1,CHROMOSOME_X,8303,10303,-,TSS_q3,X
2,CHROMOSOME_X,23953,25953,-,TSS_q2,X
3,CHROMOSOME_X,33439,35439,-,TSS_q2,X
4,CHROMOSOME_X,38220,40220,+,TSS_q1,X
5,CHROMOSOME_X,13411566,13413566,+,TSS_q2,X
6,CHROMOSOME_X,8783527,8785527,+,TSS_q2,X
7,CHROMOSOME_X,5465582,5467582,+,TSS_q4,X
8,CHROMOSOME_X,12467066,12469066,+,TSS_q3,X
9,CHROMOSOME_X,4189868,4191868,+,TSS_q4,X


/Data1/reference/X.bed has been compressed successfully to /Data1/reference/X.bed.gz
Index created successfully for /Data1/reference/X.bed.gz.tbi
| temp_bed | first 5, random 5 and last 5 out of total 12990 rows.


,chromosome,start,end,strand,type,chr-type
0,CHROMOSOME_I,9232,11232,-,TSS_q3,Autosome
1,CHROMOSOME_I,9413,11413,+,TSS_q3,Autosome
2,CHROMOSOME_I,25781,27781,-,TSS_q3,Autosome
3,CHROMOSOME_I,34350,36350,-,TSS_q4,Autosome
4,CHROMOSOME_I,42733,44733,+,TSS_q2,Autosome
5,CHROMOSOME_I,10787879,10789879,+,TSS_q4,Autosome
6,CHROMOSOME_V,9546086,9548086,+,TSS_q4,Autosome
7,CHROMOSOME_IV,17489489,17491489,-,TSS_q3,Autosome
8,CHROMOSOME_V,18461797,18463797,+,TSS_q2,Autosome
9,CHROMOSOME_II,127706,129706,-,TSS_q1,Autosome


/Data1/reference/Autosome.bed has been compressed successfully to /Data1/reference/Autosome.bed.gz
Index created successfully for /Data1/reference/Autosome.bed.gz.tbi
Saved the following bedfiles: ['/Data1/reference/X.bed.gz', '/Data1/reference/Autosome.bed.gz']
| modkit_bed_df | first 5, random 5 and last 5 out of total 9156 rows.


,0,1,2,3,4,5
0,CHROMOSOME_I,42733,44733,.,.,+
1,CHROMOSOME_I,42733,44733,.,.,-
2,CHROMOSOME_I,63021,65021,.,.,-
3,CHROMOSOME_I,63021,65021,.,.,+
4,CHROMOSOME_I,90380,92380,.,.,+
5,CHROMOSOME_X,4523221,4525221,.,.,+
6,CHROMOSOME_V,127009,129009,.,.,+
7,CHROMOSOME_X,7891389,7893389,.,.,-
8,CHROMOSOME_X,14089380,14091380,.,.,-
9,CHROMOSOME_X,10122404,10124404,.,.,+


Program finished!
['/Data1/seq_data/TubeY9B_N2_fiberseq_timec_mpx_3_21_2023/demux/mod_mappings_barcode_08_Ya_ambig.bam', '/Data1/seq_data/TubeD1a_N2_Fiberseq_Hia5_MSssI_12_22_22/basecalls/m6A/mod_mappings.sorted_Ya_ambig.bam', '/Data1/seq_data/TubeAD1_N2_fiberseq_6_13_23/mod_basecalls/mod_mappings.sorted.bam', '/Data1/seq_data/TubeAB_FiberSeq_TimeC_N2_021Aux_4_10_23/basecalls/m6A_full/demux/mod_mappings_barcode_05_Ya_ambig.bam', '/Data1/seq_data/TubeAB_FiberSeq_TimeC_N2_021Aux_4_10_23/basecalls/m6A_full/demux/mod_mappings_barcode_04_Ya_ambig.bam', '/Data1/seq_data/AH_N2_SDC2aid_AuxRem_fiberseq_8_19_23/8_20_23_run/pod5_pass/barcode07/basecalls/barcode07.mod_mappings.sorted.bam', '/Data1/seq_data/AH_N2_SDC2aid_AuxRem_fiberseq_8_19_23/8_20_23_run/pod5_pass/barcode08/basecalls/barcode08.mod_mappings.sorted.bam', '/Data1/seq_data/TubeH1_021_SDC2-AIDpAux_Hia5_MSssI_12_19/basecalls/m6A/mod_mappings.sorted_Ya_ambig.bam', '/Data1/seq_data/TubeAB_FiberSeq_TimeC_N2_021Aux_4_10_23/basecalls/m6A_fu

In [7]:
### Extracting per read modifications
out_file_names = [output_stem + "modkit-extract-" + each_condition +"_"+ str(round(each_thresh,2))+"_"+str(each_index)+ "_"+str(each_bamfrac)+
                  # convert the first 3 characters of each element in "type_selected" into a single string separated by "-"
                  "-".join([str(x)[0:7] for x in type_selected])+"_"+
                  # convert first character and the last 3 characters of each element in "choromosome_selected" into a single string separated by "-"
                  "-".join([str(x)[0]+str(x)[-3:] for x in chromosome_selected])+"_"+
                  ".bed"
                  for each_condition,each_thresh,each_index, each_bamfrac in zip(conditions,thresh_list,sample_indices,bam_fracs)]

modkit_bed_df = pd.read_csv(modkit_bed_name,sep='\t',header=None)
### Define bed file for modkit

# Function to run a single extract command
def modkit_extract(args):
    each_bam, each_thresh, each_condition, each_index, each_bamfrac,modkit_path, output_stem, modkit_bed_name, bed_window = args

    each_output = output_stem + "modkit-extract-" + each_condition +"_"+ str(round(each_thresh,2))+"_"+str(each_index)+ "_"+str(each_bamfrac)+ "-".join([str(x)[0:7] for x in type_selected])+"_"+ "-".join([str(x)[0]+str(x)[-3:] for x in chromosome_selected])+"_"+ ".bed"

    ### NOTE: Name of pileup file is not based on configurations
    ### TODO: Name of output file should be based on configs so that we aren't recomputing pileups withidentical conditions.

    # If each_output exsits, skip
    if os.path.exists(each_output):
        print(f"Skipping: {each_output}")
        return

    print(f"Starting on: {each_bam}")
    command = [
        modkit_path,
        "extract",
        "--threads",
        "20",
        "--force",
        "--mapped",
        "--ignore",
        "m",
        "--include-bed",
        modkit_bed_name,
        "--log-filepath",
        each_output + each_condition + "_modkit-extract.log",
        each_bam,
        each_output
    ]
    subprocess.run(command, text=True)

    # Create a list of arguments for each task
task_args = list(zip(
    new_bam_files,
    thresh_list,
    conditions,
    sample_indices,
    bam_fracs,
    [modkit_path]*len(new_bam_files),
    [output_stem]*len(new_bam_files),
    [modkit_bed_name]*len(new_bam_files),
    [bed_window]*len(new_bam_files)
))

# Execute commands in parallel
with Pool() as pool:
    pool.map(modkit_extract, task_args)

print("finished with:")
print(out_file_names)

Skipping: /Data1/seq_data/combined_fiber_dimelo_8_30_23/modkit-extract-N2_fiber_0.7_4_1TSS_q4-TSS_q3-TSS_q2-TSS_q1_CE_I-C_II-CIII-C_IV-CE_V-CE_X_.bedSkipping: /Data1/seq_data/combined_fiber_dimelo_8_30_23/modkit-extract-N2_fiber_0.7_3_1TSS_q4-TSS_q3-TSS_q2-TSS_q1_CE_I-C_II-CIII-C_IV-CE_V-CE_X_.bedStarting on: /Data1/seq_data/AH_N2_SDC2aid_AuxRem_fiberseq_8_19_23/8_20_23_run/pod5_pass/barcode08/basecalls/barcode08.mod_mappings.sorted.bamSkipping: /Data1/seq_data/combined_fiber_dimelo_8_30_23/modkit-extract-N2_fiber_0.7_2_1TSS_q4-TSS_q3-TSS_q2-TSS_q1_CE_I-C_II-CIII-C_IV-CE_V-CE_X_.bedSkipping: /Data1/seq_data/combined_fiber_dimelo_8_30_23/modkit-extract-N2_fiber_0.7_5_1TSS_q4-TSS_q3-TSS_q2-TSS_q1_CE_I-C_II-CIII-C_IV-CE_V-CE_X_.bedSkipping: /Data1/seq_data/combined_fiber_dimelo_8_30_23/modkit-extract-N2_fiber_0.7_1_1TSS_q4-TSS_q3-TSS_q2-TSS_q1_CE_I-C_II-CIII-C_IV-CE_V-CE_X_.bedSkipping: /Data1/seq_data/combined_fiber_dimelo_8_30_23/modkit-extract-N2_fiber_0.7_0_1TSS_q4-TSS_q3-TSS_q2-TSS_q

> > specifying include-only BED outputs only mapped sitesspecifying include-only BED outputs only mapped sites

> > parsing BED at parsing BED at modkit_temp.bedmodkit_temp.bed

> processed 9156 BED lines
> processed 9156 BED lines
> found BAM index, processing reads in 100000 base pair chunks
> found BAM index, processing reads in 100000 base pair chunks
> processed 65490 reads, 1274670 rows, skipped ~76354 reads, failed ~0 reads
Process ForkPoolWorker-28:
Process ForkPoolWorker-45:
Process ForkPoolWorker-44:
Process ForkPoolWorker-25:
Process ForkPoolWorker-47:
Process ForkPoolWorker-42:
Process ForkPoolWorker-46:
Process ForkPoolWorker-26:
Process ForkPoolWorker-37:
Process ForkPoolWorker-43:
Process ForkPoolWorker-40:
Process ForkPoolWorker-48:
Process ForkPoolWorker-41:
Process ForkPoolWorker-39:
Process ForkPoolWorker-38:
Process ForkPoolWorker-36:
Process ForkPoolWorker-30:
Process ForkPoolWorker-33:
Process ForkPoolWorker-32:
Process ForkPoolWorker-29:
Process ForkPoolWorker-31

KeyboardInterrupt: 

In [2]:
### Add bed and condition details to modkit output for plotting
# Using DataFrame merging to achieve the task without explicit loops
## Looks up the bed_start and bed_end values for each row in bedmethyl_df
def add_bed_columns_no_loops(bedmethyl_df, bed_df):

    # Initialize an empty DataFrame to store the merged data
    merged_data = pd.DataFrame()
    filtered_df = pd.DataFrame()

    # drop all rows from bedmethly_df where mod_strand != '+' or '-'
    bedmethyl_df = bedmethyl_df[(bedmethyl_df['mod_strand'] == '+') | (bedmethyl_df['mod_strand'] == '-')]
    # Get all unique chromosome-strand combinations without headers in bedmethyl_df
    unique_chrom_strand_combinations = bedmethyl_df[['chrom']].drop_duplicates().values #, 'mod_strand'
    for chrom in unique_chrom_strand_combinations: #, strand
        #print("Starting on:", chrom, strand)
        bed_subset = bed_df[(bed_df['chrom'] == chrom[0])] #& (bed_df['bed_strand'] == strand)]

        if bed_subset.empty:
            #print("No bed entries for {}".format(chrom[0]))
            # then skip this chromosome-strand combination
            continue
        #display(bed_subset.head(10))
        # Subset data for the current chromosome
        bedmethyl_subset = bedmethyl_df[(bedmethyl_df['chrom'] == chrom[0])]
        #print("bedmethyl_subset:",bedmethyl_subset.head(10))
        #display(bedmethyl_subset.head(10))
        # Explicitly cast to numeric data type
        bedmethyl_subset['ref_position'] = bedmethyl_subset['ref_position'].astype('int64')
        bed_subset['bed_start'] = bed_subset['bed_start'].astype('int64')

        # Perform the merge for the current subset
        merged_subset = pd.merge_asof(
            bedmethyl_subset,
            bed_subset[['bed_start', 'bed_end', 'bed_strand','chr_type','type']],  # Exclude 'chrom' from right DF
            left_on='ref_position',
            right_on='bed_start',
            direction='nearest'
        )

        # Append the merged data to the overall result
        merged_data = merged_data.append(merged_subset)

    #print("bedmethyl_df:")
    #display(merged_data.head(10))
    # Filter out rows where ref_position is not within bed_start and bed_end
    filtered_df = merged_data.loc[
        (merged_data['ref_position'] >= merged_data['bed_start']) &
        (merged_data['ref_position'] <= merged_data['bed_end'])
    ]
    # Filter out rows where bed_start is not a number
    filtered_df = filtered_df[filtered_df['bed_start'].notna()]
    filtered_df.reset_index(drop=True, inplace=True)

    # Print number rows in filtered_df
    print("Found {} rows in filtered_df".format(len(filtered_df)))

    #display(filtered_df)
    return filtered_df

# Initialize comb_readmethyl_df as an empty dataframe
comb_readmethyl_df = pd.DataFrame()

# Build combined bed df
combined_bed_df=pd.DataFrame()
for each_bed in new_bed_files:
    bed_path = each_bed[:-3] # remove .gz
    print("Starting on:",bed_path)
    # read bedpath and append to bed_df
    combined_bed_df = combined_bed_df.append(pd.read_csv(bed_path, sep="\t", header=None))#skiprows=1))
combined_bed_df.columns = ['chrom','bed_start','bed_end','bed_strand','type','chr_type']

combined_bed_df.sort_values(['chrom','bed_start'], inplace=True)
combined_bed_df.reset_index(drop=True, inplace=True)
display(combined_bed_df.head(10))

# Initialize comb_bedmethyl_plot_df
comb_bedmethyl_plot_df = pd.DataFrame()

# Create combined plotting dataframe
for each_output,each_condition in zip(out_file_names,conditions):
    print("Starting on:",each_output)
    # Define bed methyl columns and import bedmethyl file
    bedmethyl_df = pd.DataFrame()
    bedmethyl_cols = ['read_id',
    'forward_read_position',
    'ref_position',
    'chrom',
    'mod_strand',
    'ref_strand',
    'ref_mod_strand',
    'fw_soft_clipped_start',
    'fw_soft_clipped_end',
    'read_length',
    'mod_qual',
    'mod_code',
    'base_qual',
    'ref_kmer',
    'query_kmer',
    'canonical_base',
    'modified_primary_base',
    'inferred']
    bedmethyl_df=pd.read_csv(each_output, sep="\t", header=None, names=bedmethyl_cols,skiprows=1)

    # sort bedmethyl_df by chrom and start_position, required for merging in bed matching.
    bedmethyl_df.sort_values(['chrom','ref_position'], inplace=True)
    bedmethyl_df.reset_index(drop=True, inplace=True)
    #display(bedmethyl_df.head(10))

    # Adding new columns to bedmethyl_df using condition matching
    bedmethyl_df = add_bed_columns_no_loops(bedmethyl_df.copy(), combined_bed_df)
    # Add rel_start and rel_end columns equal to start-bed_start and end-bed_start
    bedmethyl_df['rel_pos'] = bedmethyl_df['ref_position'] - bedmethyl_df['bed_start'] -bed_window +1

    # add condition column
    bedmethyl_df['condition'] = each_condition

    # if comb_bedmethyl_plot_df is null, set it equal to bedmethyl_plot
    if comb_bedmethyl_plot_df.empty:
        print("comb_bedmethyl_plot_df is empty, setting it equal to bedmethyl_plot...")
        comb_bedmethyl_plot_df = bedmethyl_df.copy()
    # else append bedmethyl_plot to comb_bedmethyl_plot_df
    else:
        print("comb_bedmethyl_plot_df is not empty, appending bedmethyl_plot...")
        comb_bedmethyl_plot_df = comb_bedmethyl_plot_df.append(bedmethyl_df)

if any(x in type_selected[0] for x in ("TSS", "TES", "MEX")):
    print("Strand orientation sensitive type selected, multiplying rel_start by -1 for '-' strand genes...")
    mask = comb_bedmethyl_plot_df['bed_strand'] == '-'
    comb_bedmethyl_plot_df.loc[mask, 'rel_pos'] *= -1
    #comb_bedmethyl_plot_df.sort_values(['chrom','rel_pos'], inplace=True)

# Set entire mod_qual column equal to 1 base on m6A thresh
threshold = m6A_thresh / 255
# convert comb_bedmethly_plot_df_final['mod_qual'] to a float
comb_bedmethyl_plot_df['mod_qual'] = comb_bedmethyl_plot_df['mod_qual'].astype(float)
comb_bedmethyl_plot_df['mod_qual_bin'] = np.where(comb_bedmethyl_plot_df['mod_qual'] > threshold, 1, 0)

#add read_start and read_end columns to comb_bedmethyl_plot_df_final based on min and max ref_position for each read_id
print("Calculating read start and end...")
# Reduce the DataFrame size by selecting only the columns you need
small_df = comb_bedmethyl_plot_df[['read_id', 'rel_pos']]
# Perform the grouping and aggregation in one step
grouped_df = small_df.groupby('read_id')['rel_pos'].agg(['min', 'max']).reset_index()
# Rename the columns
grouped_df.rename(columns={'min': 'rel_read_start', 'max': 'rel_read_end'}, inplace=True)
# Merge the aggregated results back to the original DataFrame
comb_bedmethyl_plot_df = pd.merge(comb_bedmethyl_plot_df, grouped_df, on='read_id', how='left')
# delete small_df and grouped_df
del small_df
del grouped_df

#comb_bedmethyl_plot_df.sort_values(['chrom','rel_pos'], inplace=True)
comb_bedmethyl_plot_df = comb_bedmethyl_plot_df[comb_bedmethyl_plot_df['bed_start'].notna()]
comb_bedmethyl_plot_df.reset_index(inplace=True, drop=True)

# print count of unique read_ids for each condition
print("All unique read_ids for each condition:")
print(comb_bedmethyl_plot_df.groupby(['condition'])['read_id'].nunique())

#display(comb_bedmethyl_plot_df.head(10))
# drop all rows where read_length is less than 1000
comb_bedmethyl_plot_df = comb_bedmethyl_plot_df[comb_bedmethyl_plot_df['read_length'] >= 1000]


# print count of unique read_ids for each condition
print("All unique read_ids for each condition > read_len of 1000:")
print(comb_bedmethyl_plot_df.groupby(['condition'])['read_id'].nunique())

# Define SQLite database file name
#db_fn = "temp_files/" + "plot_db_" + "-".join([str(x)[:6] for x in type_selected]) + "_" + str(round(thresh_list[0],2)) + "_" + str(bam_fracs[0]) + ".db"
# Create a SQLite database connection
#conn = sqlite3.connect(db_fn)
# Save the DataFrame to SQLite database
#comb_bedmethyl_plot_df.to_sql('bedmethyl_plot', conn, if_exists='replace', index=False)
# Close the connection
#conn.close()

NameError: name 'pd' is not defined

In [1]:
# Similar to other dataframes, define fn based on configurations. If it doesn't exist, create it, otherwise import it.
plot_df_fn = "temp_files/" + "plot_df_" + "-".join([str(x)[0:12] for x in type_selected])+"_"+ str(thresh_list[0])+"_"+str(bam_fracs[0])+ ".csv"
comb_bedmethyl_plot_df.to_csv(plot_df_fn, index=False)
if os.path.exists(plot_df_fn):
    print("plot_df_fn exists, importing...")
    plot_df = pd.read_csv(plot_df_fn)
else:
    print("plot_df_fn does not exist, creating...")
    comb_bedmethyl_plot_df.to_csv(plot_df_fn, index=False)
    plot_df = comb_bedmethyl_plot_df.copy()

NameError: name 'type_selected' is not defined